In [1]:
import itertools
from concurrent.futures.thread import ThreadPoolExecutor
import requests
import pandas as pd
from sqlalchemy import MetaData, create_engine
import networkx as nx
import math

# System Data

In [2]:
systemsquery = '''
create table systems
(
	system_id BIGINT
		constraint systems_pk
			primary key,
	name TEXT,
	constellation_id BIGINT
		constraint systems_constellations_constellation_id_fk
			references constellations,
	security_status FLOAT,
	x FLOAT,
	y FLOAT,
	z FLOAT
);
'''
constellationsquery = '''
create table constellations
(
	constellation_id BIGINT
		constraint constellations_pk
			primary key,
	name TEXT,
	region_id BIGINT
		constraint constellations_regions_region_id_fk
			references regions
);
'''
regionsquery = '''
create table regions
(
	region_id BIGINT
		constraint regions_pk
			primary key,
	name TEXT
)
'''
systemkillsquery = '''
create table system_kills
(
	system_id BIGINT
		constraint system_kills_pk
			primary key
		constraint system_kills_systems_system_id_fk
			references systems (system_id),
	npc_kills BIGINT,
	ship_kills BIGINT,
	pod_kills BIGINT
)
'''
theraquery = '''
create table thera
(
	system_id BIGINT
		constraint thera_pk
			primary key
		constraint thera_systems_system_id_fk
			references systems (system_id),
	type TEXT
)
'''
stargatesquery = '''
create table stargates
(
	gate_id BIGINT
		constraint stargates_pk
			primary key
)
'''
connectionsquery = '''
create table connections
(
	source_id BIGINT
		constraint connections_systems_system_id_fk
			references systems (system_id),
	destination_id BIGINT
		constraint connections_systems_system_id_fk_2
			references systems (system_id)
)
'''

engine = create_engine('sqlite:///eve.db')
connection = engine.connect()
metadata = MetaData()

connection.execute(regionsquery)
connection.execute(constellationsquery)
connection.execute(systemsquery)
connection.execute(systemkillsquery)
connection.execute(theraquery)
connection.execute(stargatesquery)
connection.execute(connectionsquery)

In [3]:
def system(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/systems/{id}/?datasource=tranquility&language=en').json()
    systemdict = {}
    systemdict['system_id'] = data['system_id']
    systemdict['name'] = data['name']
    systemdict['constellation_id'] = data['constellation_id']
    systemdict['security_status'] = data['security_status']
    systemdict['x'] = data['position']['x']
    systemdict['y'] = data['position']['y']
    systemdict['z'] = data['position']['z']
    return systemdict

def getsystems(systems):
    with ThreadPoolExecutor() as tp:
        results = tp.map(system,systems)
        return [result for result in results]


r = requests.get('https://esi.evetech.net/latest/universe/systems/?datasource=tranquility')
systems = r.json()
syslist = getsystems(systems)
assert len(syslist) == len(systems)
df = pd.DataFrame(syslist)
df.to_sql('systems', con = connection, index = False, if_exists='append')

# Constellations

In [4]:
def constellation(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/constellations/{id}/?datasource=tranquility&language=en').json()
    constellationdict = {}
    constellationdict['constellation_id'] = data['constellation_id']
    constellationdict['name'] = data['name']
    constellationdict['region_id'] = data['region_id']
    return constellationdict

def getconstellations(constellations):
    with ThreadPoolExecutor() as tp:
        results = tp.map(constellation,constellations)
        return [result for result in results]


r = requests.get('https://esi.evetech.net/latest/universe/constellations/?datasource=tranquility')
constellations = r.json()
constlist = getconstellations(constellations)
assert len(constlist) == len(constellations)
df = pd.DataFrame(constlist)
df.to_sql('constellations', con = connection, index = False, if_exists='append')

# Regions

In [5]:
def region(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/regions/{id}/?datasource=tranquility&language=en').json()
    regiondict = {}
    regiondict['region_id'] = data['region_id']
    regiondict['name'] = data['name']
    return regiondict

def getregions(regions):
    with ThreadPoolExecutor() as tp:
        results = tp.map(region,regions)
        return [result for result in results]


r = requests.get('https://esi.evetech.net/latest/universe/regions/?datasource=tranquility')
regions = r.json()
reglist = getregions(regions)
assert len(reglist) == len(regions)
df = pd.DataFrame(reglist)
df.to_sql('regions', con = connection, index = False, if_exists='append')

# System Kills

In [6]:
data = requests.get('https://esi.evetech.net/latest/universe/system_kills/?datasource=tranquility').json()
df = pd.DataFrame(data)
df = df.reindex(columns = ['system_id','npc_kills','ship_kills','pod_kills'])
df.to_sql('system_kills', con = connection, index= False, if_exists = 'append')

# Thera Connections

In [7]:
data = requests.get('https://www.eve-scout.com/api/wormholes').json()
theradata = []
for tdict in data:
    theradata.append({
        'system_id':tdict['destinationSolarSystem']['id'],
        'type':tdict['sourceWormholeType']['name']
     })
df = pd.DataFrame(theradata)
connection.execute('DELETE from thera')
df.to_sql('thera', con = connection, index = False, if_exists='append')

# Stargates

In [8]:
def stargate(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/systems/{id}/?datasource=tranquility&language=en').json()
    try:
        return data['stargates']
    except KeyError:
        return []

def getstargates(stargates):
    with ThreadPoolExecutor() as tp:
        results = tp.map(stargate,stargates)
        rlist = [result for result in results]
        return list(itertools.chain(*rlist))


r = requests.get('https://esi.evetech.net/latest/universe/systems/?datasource=tranquility')
stargates = r.json()
gatelist = getstargates(stargates)
df = pd.DataFrame(gatelist)
df.columns = ['gate_id']
df.to_sql('stargates', con = connection, index = False, if_exists='append')

# Connections


In [9]:
def sconnection(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/stargates/{id}/?datasource=tranquility').json()
    return {
        'source_id':data['system_id'],
        'destination_id':data['destination']['system_id']
    }

def getconnections(stargates):
    with ThreadPoolExecutor() as tp:
        results = tp.map(sconnection,stargates)
        return [result for result in results]


stargates = connection.execute('select gate_id from stargates').fetchall()
stargates = [y for x in stargates for y in x]
conlist = getconnections(stargates)
assert len(conlist) == len(stargates)
df = pd.DataFrame(conlist)
df.to_sql('connections', con = connection, index = False, if_exists='append')

In [10]:
noduplicates = connection.execute('SELECT DISTINCT source_id, destination_id FROM connections t1 WHERE t1.source_id > t1.destination_id OR NOT EXISTS (SELECT * FROM connections t2 WHERE t2.source_id = t1.destination_id AND t2.destination_id = t1.source_id)').fetchall()
df = pd.DataFrame(noduplicates)
df.columns = ['source_id','destination_id']
connection.execute('DELETE from connections')
df.to_sql('connections',con = connection, index=False, if_exists='append')

In [11]:
eve = nx.Graph()
eve.add_edges_from(noduplicates)

In [12]:
query = """
SELECT s.system_id, s.name as system, security_status, r.name as region, npc_kills, pod_kills, ship_kills, x, y, z
FROM systems AS s
         INNER JOIN constellations c ON c.constellation_id = s.constellation_id
         INNER JOIN regions r ON r.region_id = c.region_id
         INNER JOIN system_kills sk ON s.system_id = sk.system_id
         """
test = connection.execute(query)
df = pd.DataFrame(test, columns=test.keys()).sort_values(by=['npc_kills'], ascending=False)
df

,system_id,system,security_status,region,npc_kills,pod_kills,ship_kills,x,y,z
1989,30003645,O36A-P,-0.084704,Tenal,2240,0,0,-6.397321e+16,8.255295e+16,4.294112e+17
2321,30004206,4LNE-M,-0.566918,Cobalt Edge,1859,1,1,2.830588e+17,5.436335e+16,2.477010e+17
2279,30004142,Hikansog,0.811302,Kador,1314,0,1,-2.352238e+17,5.373238e+16,-1.218548e+17
195,30000242,8TPX-N,-0.437192,Vale of the Silent,1237,0,1,-7.376450e+16,1.100980e+17,1.876280e+17
2281,30004144,Yebouz,0.882179,Kador,1200,0,0,-2.347617e+17,6.249611e+16,-1.297336e+17
...,...,...,...,...,...,...,...,...,...,...
532,30001198,GE-8JV,-0.209423,Catch,0,0,7,-7.444434e+16,6.109297e+15,-2.448536e+17
2697,30005005,Ignebaener,-1.000000,Pochven,0,1,1,-2.382909e+17,5.558935e+16,4.385876e+16
530,30001196,Q-S7ZD,-0.032710,Catch,0,1,1,-4.927858e+16,3.365150e+16,-2.314420e+17
1263,30002652,Ala,-1.000000,Pochven,0,1,1,-1.747970e+17,3.572408e+16,-9.752209e+15


In [13]:
query = """
SELECT thera.system_id, name, x, y, z from thera
INNER JOIN systems s ON s.system_id = thera.system_id
"""
theradata = connection.execute(query)
thera = pd.DataFrame(theradata, columns= theradata.keys())
thera

,system_id,name,x,y,z
0,30000203,Eruka,-9.521840e+16,6.535627e+16,1.433769e+17
1,30000266,Z-8Q65,-7.485899e+16,1.006970e+17,2.169115e+17
2,30000717,Q-NA5H,1.023382e+17,1.527003e+16,-1.285482e+17
3,30001299,D-SKWC,-1.882945e+17,1.141675e+17,2.885838e+17
4,30001308,S-1ZXZ,-1.476756e+17,1.091289e+17,3.575399e+17
5,30001387,Isikano,-1.630471e+17,9.340016e+16,1.391321e+17
6,30002083,Uisper,-1.294856e+17,2.851737e+16,3.070949e+16
7,30002096,Hofjaldgund,-1.320439e+17,1.900875e+16,1.459811e+16
8,30002217,Hutian,-1.994459e+17,6.876697e+16,-1.047576e+17
9,30002865,LEM-I1,1.091623e+17,7.145899e+16,8.188988e+16


In [14]:
closest_gate = []
for row in df.itertuples():
    shortest = int(10000000)
    closest = int(0)
    for trow in thera.itertuples():
        try:
            pathlength = len(nx.shortest_path(eve,source = row.system_id, target = trow.system_id))
            if shortest > pathlength:
                shortest = pathlength
                closest = trow.name
        except:
            continue
    closest_gate.append(
        {
            'closest_gate': closest,
            'gate_distance': shortest
        }
    )

In [15]:
len(closest_gate)
df['closest_gate'] = [x['closest_gate'] for x in closest_gate]
df['gate_distance'] = [x['gate_distance'] for x in closest_gate]
df

,system_id,system,security_status,region,npc_kills,pod_kills,ship_kills,x,y,z,closest_gate,gate_distance
1989,30003645,O36A-P,-0.084704,Tenal,2240,0,0,-6.397321e+16,8.255295e+16,4.294112e+17,Z-8Q65,15
2321,30004206,4LNE-M,-0.566918,Cobalt Edge,1859,1,1,2.830588e+17,5.436335e+16,2.477010e+17,Z-8Q65,16
2279,30004142,Hikansog,0.811302,Kador,1314,0,1,-2.352238e+17,5.373238e+16,-1.218548e+17,Kooreng,9
195,30000242,8TPX-N,-0.437192,Vale of the Silent,1237,0,1,-7.376450e+16,1.100980e+17,1.876280e+17,Z-8Q65,5
2281,30004144,Yebouz,0.882179,Kador,1200,0,0,-2.347617e+17,6.249611e+16,-1.297336e+17,Kooreng,11
...,...,...,...,...,...,...,...,...,...,...,...,...
532,30001198,GE-8JV,-0.209423,Catch,0,0,7,-7.444434e+16,6.109297e+15,-2.448536e+17,C-FD0D,13
2697,30005005,Ignebaener,-1.000000,Pochven,0,1,1,-2.382909e+17,5.558935e+16,4.385876e+16,0,10000000
530,30001196,Q-S7ZD,-0.032710,Catch,0,1,1,-4.927858e+16,3.365150e+16,-2.314420e+17,C-FD0D,7
1263,30002652,Ala,-1.000000,Pochven,0,1,1,-1.747970e+17,3.572408e+16,-9.752209e+15,0,10000000


In [16]:
def distance(curloc, otherloc):
    dist = math.sqrt(
        (curloc[0] - otherloc[0]) ** 2 + (curloc[1] - otherloc[1]) ** 2 + (curloc[2] - otherloc[2]) ** 2)
    return dist / 149597870691 / 63239.6717


def closestly(indf,thera):
    dists = []
    names = []
    for tup in indf.itertuples():
        tempdist = []
        tempname = []
        for tup2 in thera.itertuples():
            tempdist.append(distance((tup.x, tup.y, tup.z), (tup2.x, tup2.y, tup2.z)))
            tempname.append(tup2.name)
        smallest = tempdist.index(min(tempdist))
        dists.append(tempdist[smallest])
        names.append(tempname[smallest])

    return names, dists

In [17]:
df['closest_ly'], df['distance_ly'] = closestly(df,thera)
df[['system','region','npc_kills','pod_kills','ship_kills','closest_gate','gate_distance','closest_ly','distance_ly']]



,system,region,npc_kills,pod_kills,ship_kills,closest_gate,gate_distance,closest_ly,distance_ly
1989,O36A-P,Tenal,2240,0,0,Z-8Q65,15,S-1ZXZ,11.995180
2321,4LNE-M,Cobalt Edge,1859,1,1,Z-8Q65,16,LEM-I1,25.462134
2279,Hikansog,Kador,1314,0,1,Kooreng,9,Kooreng,4.323074
195,8TPX-N,Vale of the Silent,1237,0,1,Z-8Q65,5,Z-8Q65,3.252999
2281,Yebouz,Kador,1200,0,0,Kooreng,11,Hutian,4.619967
...,...,...,...,...,...,...,...,...,...
532,GE-8JV,Catch,0,0,7,C-FD0D,13,C-FD0D,9.055654
2697,Ignebaener,Pochven,0,1,1,0,10000000,Yvangier,4.698334
530,Q-S7ZD,Catch,0,1,1,C-FD0D,7,C-FD0D,7.421912
1263,Ala,Pochven,0,1,1,0,10000000,Fahruni,3.004714
